## Comprovació absència NA i Outliers df

In [1]:
import pyspark
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import col, expr, lit
import duckdb

In [ ]:
sc = SparkSession.builder.getOrCreate()

In [21]:
paths = ['data_lake/compravenda_sup.parquet', 'data_lake/rent_price.parquet', 'data_lake/renda.parquet']
col_numeriques = {'data_lake/compravenda_sup.parquet': ['Nombre']
                  , 'data_lake/renda.parquet': ['Import_Euros']
                , 'data_lake/rent_price.parquet': ['Price']}

In [31]:
for path in paths:
    RDD = sc.read.parquet(path)
    
    # Verificar si hi ha valors NA
    na_count = RDD.na.drop().count()
    if na_count == RDD.count():
        print(f"No hi ha valors NA al DataFrame {path}")
    else:
        print(f"Hi ha {RDD.count() - na_count} valors NA al DataFrame {path}")

    # Verificar si hi ha valors duplicats
    RDD.distinct()
    print(f"Hi ha {RDD.count() - RDD.distinct().count()} valors duplicats al DataFrame {path} de {RDD.count()} valors totals")
    
    for columna in col_numeriques[path]:
        valor_min = RDD.rdd.filter(lambda f: '-' not in str(f[columna]))\
            .map(lambda x: (1, x[columna]))\
            .mapValues(lambda x: float(x) if type(x) == str else x)\
            .reduceByKey(lambda x, y: min(x,y)).collect()[0][1]
        valor_max = RDD.rdd.filter(lambda f: '-' not in str(f[columna]))\
            .map(lambda x: (1, float(x[columna])))\
            .reduceByKey(lambda x, y: max(x,y)).collect()[0][1]
        
        RDD = RDD.withColumn(columna, (RDD[columna] - valor_min)/ (valor_max - valor_min))
        
        # IQR método
        IQR = RDD.approxQuantile(columna, [0.25, 0.75], 0.001)
        IQR_values = IQR[1] - IQR[0]
        
        RDD = RDD.withColumn('IQR', lit(IQR_values))
                
        outliersremoved = RDD.filter(col(columna) >= IQR[0] - 1.5 * IQR_values)\
            .filter(col(columna) <= IQR[1] + 1.5 * IQR_values)
        print(f"El DataFrame {path} ha passat de tenir {RDD.count()} a {outliersremoved.count()} valors sense outliers")
    #outliersremoved.write.db("ruta_de_salida/" + path)

No hi ha valors NA al DataFrame data_lake/compravenda_sup.parquet
Hi ha 0 valors duplicats al DataFrame data_lake/compravenda_sup.parquet de 876 valors totals
El DataFrame data_lake/compravenda_sup.parquet ha passat de tenir 876 a 518 valors sense outliers
No hi ha valors NA al DataFrame data_lake/rent_price.parquet
Hi ha 0 valors duplicats al DataFrame data_lake/rent_price.parquet de 4622 valors totals
El DataFrame data_lake/rent_price.parquet ha passat de tenir 4622 a 4617 valors sense outliers
No hi ha valors NA al DataFrame data_lake/renda.parquet
Hi ha 0 valors duplicats al DataFrame data_lake/renda.parquet de 1068 valors totals
El DataFrame data_lake/renda.parquet ha passat de tenir 1068 a 1001 valors sense outliers


In [3]:
spark = SparkSession.builder\
    .config("spark.jars", "duckdb.jar") \
    .getOrCreate()

24/04/16 15:58:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [82]:
# rent_price
DF = spark.read \
  .format("jdbc") \
  .option("url", "jdbc:duckdb:formatted_zone/rent_price.db") \
  .option("driver", "org.duckdb.DuckDBDriver") \
  .option("query", "SELECT * FROM RDD") \
  .load()

# verificar duplicats
print(f"Hi ha {DF.count() - DF.distinct().count()} valors duplicats al DataFrame 'formatted_zone/rent_price.db' de {DF.count()} valors totals")

# verificar NA
print(f'Hi ha {DF.count() - DF.na.drop().count()} NAs')

# escalar obteninint mínim i màxims via query en funció de la unitat
min_max = spark.read \
  .format("jdbc") \
  .option("url", "jdbc:duckdb:formatted_zone/rent_price.db") \
  .option("driver", "org.duckdb.DuckDBDriver") \
  .option("query", 'SELECT "Average _rent", min(Price) AS min_price , max(Price) AS max_price FROM RDD GROUP BY "Average _rent"') \
  .load().collect()

DF = DF.withColumn('Price', F.when(DF['Average _rent'] == min_max[0][0], (DF['Price'] - min_max[0].min_price)/(min_max[0].max_price - min_max[0].min_price)).otherwise((DF['Price'] - min_max[1].min_price)/(min_max[1].max_price - min_max[1].min_price)))

# outliers
IQR = DF.approxQuantile('Price', [0.25, 0.75], 0.001)
IQR_values = IQR[1] - IQR[0]
        
DF = DF.withColumn('IQR', lit(IQR_values))
                
not_outliers = DF.filter(col('Price') >= IQR[0] - 1.5 * IQR_values)\
            .filter(col('Price') <= IQR[1] + 1.5 * IQR_values)

outliers = DF.subtract(not_outliers)
print(f"Hi ha {DF.count()} a {not_outliers.count()} valors sense outliers")

Hi ha 0 valors duplicats al DataFrame 'formatted_zone/rent_price.db' de 4622 valors totals
Hi ha 0 NAs


In [90]:
DF.subtract(outliersremoved).count()

36

In [50]:
DF = DF.withColumn('Price', (DF['Price'] - unitat[DF['Average _rent']][0])/ (unitat[DF['Average _rent']][0][1] - unitat[DF['Average _rent']][0]))

TypeError: unhashable type: 'Column'

In [40]:
test = spark.read \
  .format("jdbc") \
  .option("url", "jdbc:duckdb:formatted_zone/rent_price.db") \
  .option("driver", "org.duckdb.DuckDBDriver") \
  .option("query", 'SELECT "Average _rent", min(Price) AS min_price , max(Price) AS max_price FROM RDD GROUP BY "Average _rent"') \
  .load()

In [52]:
test.show()

+--------------------+---------+---------+
|       Average _rent|min_price|max_price|
+--------------------+---------+---------+
|average rent per ...|     3.18|     21.3|
|average rent (eur...|   142.34|   2034.0|
+--------------------+---------+---------+

